In [ ]:
# Import earthengine API
import ee
# Authenticate and initialise
ee.Authenticate()
ee.Initialize(project="ee-encaramelo")

In [ ]:
# Load S2 (L1C) image in area of interest
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

# Image from the 10th May 2022
location = ee.Geometry.Point(-0.809149,37.715081)
col = s2.filterDate('2022-05-09', '2022-05-11').filterBounds(location)
print("Collection size:", col.size().getInfo())
image = s2.filterDate('2022-05-09', '2022-05-11').filterBounds(location).first()

In [ ]:
# Load S2 (L2A - sen2cor) image in area of interest
s2_l2a = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Image from the 10th May 2022
col_l2a = s2_l2a.filterDate('2022-05-09', '2022-05-11').filterBounds(location)
print("Collection size:", col_l2a.size().getInfo())
image_l2a = col_l2a.first()

In [ ]:
import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro


# Use folium to visualize the imagery.
map = folium.Map(location=[37.715081,-0.809149],zoom_start=11)
#map.setOptions('HYBRID') # To see GE map underneath

map.addLayer(image, {'min': 1500, 'max': 3000, 'bands':['B4','B3','B2']}, 'image_l1c')
map.addLayer(image_l2a, {'min': 0, 'max': 4000, 'bands':['B4','B3','B2']}, 'image_l2a')
folium.LayerControl().add_to(map)
map

In [ ]:
# Applying land mask to image
hansenImage = ee.Image('UMD/hansen/global_forest_change_2024_v1_12')

#Select the land/water mask
datamask = hansenImage.select('datamask')

# Create a binary mask
land = datamask.eq(1)
water = land.Not()

# Make an image collection of visualization images.
#map.addLayer(mask)
#map


# Mask water with itself to mask all the zeros (non-water)
#image = image.mask(water)
masked_image = image.updateMask(water)
masked_image_l2a = image_l2a.updateMask(water)

In [ ]:
# Cropping image to area of interest
box = ee.Geometry.BBox(-0.871229,37.609765, -0.677110, 37.833082)
cropped_masked_image = masked_image.clip(box)
cropped_masked_image_l2a = masked_image_l2a.clip(box)

map = folium.Map(location=[37.715081,-0.809149],zoom_start=11.5)
map.addLayer(cropped_masked_image, {'min': 1500, 'max': 3000, 'bands':['B4','B3','B2']}, 'cropped_masked_image')
map.addLayer(cropped_masked_image_l2a, {'min': 0, 'max': 4000, 'bands':['B4','B3','B2']}, 'cropped_masked_image_l2a')
folium.LayerControl().add_to(map)
map

In [ ]:
# NDVI OVER AREA

ndvi = cropped_masked_image_l2a.normalizedDifference(['B8', 'B4']).rename('NDVI') # nir <-> red

#!pip install geemap
import ee
import geemap

map = geemap.Map(center=(37.715081,-0.809149), zoom=11.5)
map.addLayer(ndvi, {min: -1, max: 1, 'palette':['blue', 'yellow']}, 'NDVI')# min: -0.09, max: -0.01
map

Now, let's look at the Normalized Difference Chlorophyll Index (NDCI), which is a metric similar to NDVI, to estimate the presence and concentration of chlorophyll-a in water bodies. The NDCI ranges from -1 to +1.

**🌊 NDCI Value Interpretation**

*   below 0: Clear water, low chlorophyll concentration;
*   0 to 0.2:	Moderate, chlorophyll levels;
* above 0.2:	High chlorophyll, potential algal blooms.

In [ ]:
# Calculate NDCI: (B5 - B4) / (B5 + B4)

ndci = cropped_masked_image_l2a.normalizedDifference(['B5', 'B4']).rename('NDCI')

In [ ]:
# Visualisation
map = geemap.Map(center=(37.715081,-0.809149), zoom=11.5)
map.addLayer(ndci, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDCI')
map